In [1]:
!python -m deeppavlov install squad_ru_rubert

2021-05-25 19:06:15.640 INFO in 'deeppavlov.core.common.file'['file'] at line 32: Interpreting 'squad_ru_rubert' as '/home/postas/anaconda3/envs/deeppavlov/lib/python3.6/site-packages/deeppavlov/configs/squad/squad_ru_rubert.json'
  Cloning https://github.com/deepmipt/bert.git (to revision feat/multi_gpu) to /tmp/pip-req-build-iystdaqw
  Running command git clone -q https://github.com/deepmipt/bert.git /tmp/pip-req-build-iystdaqw
  Created wheel for bert-dp: filename=bert_dp-1.0-py3-none-any.whl size=23580 sha256=b16ec23453bbcec8490f1f30e943857410902247c191f6850a8b9b7e0ce1c69f
  Stored in directory: /tmp/pip-ephem-wheel-cache-jhbucuwd/wheels/65/8c/cb/bb585d79f235c48068b6f4f70f99118fedc3d964d16b1e2caa
Successfully built bert-dp
     |████████████████████████████████| 110.5 MB 6.2 MB/s eta 0:00:01    |██████████████▏                 | 48.7 MB 213 kB/s eta 0:04:50
     |████████████████████████████████| 50 kB 4.0 MB/s eta 0:00:011
  Using cached termcolor-1.1.0.tar.gz (3.9 kB)
  Using cac

In [7]:
!python -m pip install transformers

  Using cached transformers-4.6.1-py3-none-any.whl (2.2 MB)
     |████████████████████████████████| 722 kB 341 kB/s eta 0:00:01
     |████████████████████████████████| 3.3 MB 173 kB/s eta 0:00:01
  Using cached huggingface_hub-0.0.8-py3-none-any.whl (34 kB)


In [1]:
from deeppavlov import build_model, configs
from deeppavlov.core.common.file import read_json
import torch
import pymorphy2
import re
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm
import numpy as np

In [2]:
from transformers import BertTokenizer, TFAutoModel, AutoTokenizer, AutoModel

In [3]:
squad_model = build_model(configs.squad.squad_ru_rubert, download=False)

[nltk_data] Downloading package punkt to /home/postas/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/postas/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package perluniprops to
[nltk_data]     /home/postas/nltk_data...
[nltk_data]   Package perluniprops is already up-to-date!
[nltk_data] Downloading package nonbreaking_prefixes to
[nltk_data]     /home/postas/nltk_data...
[nltk_data]   Package nonbreaking_prefixes is already up-to-date!












The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use keras.layers.Dense instead.
Instructions for updating:
Please use `layer.__call__` method instead.


Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.





Instructions for updating:
Use standard file APIs to check for files with this prefix.


2021-05-30 23:52:56.435 INFO in 'deeppavlov.core.models.tf_model'['tf_model'] at line 51: [loading model from /home/postas/.deeppavlov/models/squad_ru_bert/model_rubert]



INFO:tensorflow:Restoring parameters from /home/postas/.deeppavlov/models/squad_ru_bert/model_rubert


In [4]:
# Embedder

In [5]:
tokenizer = AutoTokenizer.from_pretrained("sberbank-ai/sbert_large_nlu_ru")

In [6]:
model = AutoModel.from_pretrained("sberbank-ai/sbert_large_nlu_ru")

In [7]:
encoded_input = tokenizer('химера', padding='max_length', truncation=True, max_length=20, return_tensors='pt')

In [8]:
encoded_input

{'input_ids': tensor([[  101, 43667,  4475,   102,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])}

In [9]:
with torch.no_grad():
    model_output = model(**encoded_input)

In [10]:
model_output[1]

tensor([[-0.2461,  0.3397, -0.8128,  ..., -0.2800,  0.8480, -0.1490]])

In [11]:
pdist = torch.nn.PairwiseDistance(p=8.0)

In [12]:
pdist(model_output[1], model_output[1]).item()

0.0

In [13]:
with open('datasets/qa_route_specs.txt', 'r') as source:
    text = source.readlines()

In [14]:
pattern = re.compile(r'Маршрут "(.+?)"')
names = [re.search(pattern, route).group(1) for route in text]

In [15]:
def preprocess(sentences, lemmer=pymorphy2.MorphAnalyzer()):
    return [' '.join([lemmer.parse(token)[0].normal_form.lower() for token in sent.split()]) for sent in sentences]

In [16]:
titles_tokenized = tokenizer(preprocess(names), padding='max_length',
                             truncation=True, max_length=20, return_tensors='pt')

In [17]:
class TitlesDataset(Dataset):
    def __init__(self, titles):
        self.titles = titles
        
    def __len__(self):
        return self.titles['input_ids'].shape[0]
    
    def __getitem__(self, idx):
        return {'input_ids': self.titles['input_ids'][idx], 
                'token_type_ids': self.titles['token_type_ids'][idx],
                'attention_mask': self.titles['attention_mask'][idx]}

In [18]:
dataset = TitlesDataset(titles_tokenized)
dataset.__len__()

32818

In [19]:
dataset.__getitem__(0)

{'input_ids': tensor([101, 159, 774, 378, 102,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0]),
 'token_type_ids': tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 'attention_mask': tensor([1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])}

In [20]:
encoded_input

{'input_ids': tensor([[  101, 43667,  4475,   102,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])}

In [21]:
data_loader = DataLoader(dataset, batch_size=128, shuffle=False)

In [22]:
iter(data_loader).next()

{'input_ids': tensor([[  101,   159,   774,  ...,     0,     0,     0],
         [  101, 20981,  1918,  ...,     0,     0,     0],
         [  101,  1886,   978,  ...,     0,     0,     0],
         ...,
         [  101, 55987,   102,  ...,     0,     0,     0],
         [  101, 22178,   102,  ...,     0,     0,     0],
         [  101, 64483,   102,  ...,     0,     0,     0]]),
 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0]]),
 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         ...,
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0]])}

In [23]:
dataset.__getitem__(0)

{'input_ids': tensor([101, 159, 774, 378, 102,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0]),
 'token_type_ids': tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 'attention_mask': tensor([1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])}

In [24]:
tens = iter(data_loader).next()

In [25]:
with torch.no_grad():
    model(input_ids=tens['input_ids'], attention_mask=tens['attention_mask'])

In [26]:
# torch.set_num_threads(16)

In [27]:
titles_vectorized = []
with torch.no_grad():
    for batch in tqdm(data_loader):
        titles_vectorized.append(model(**batch)[1])

100%|██████████| 257/257 [37:47<00:00,  8.82s/it]


In [159]:
# Как же это долго.. все дело видимо в том, что в этом энвайронменте нет подобающей версии cuda... 
# надо делать энв со старой кудой, что не хочется, поскольку и так куча ограничений изза диппавлова.
# На tf c GPU эта операция занимает меньше чем полминуты.
# Но по факту, работать эта штука будет на серваке, где никакой куды нет. Есть ли там вообще видеокарта и то вопрос
# Поэтому если делать это скажем раз в сутки, то не жалко, снизить количество ядер и пусть часок в сутки тратит

In [28]:
len(titles_vectorized)

257

In [29]:
titles_vectorized[0].shape

torch.Size([128, 1024])

In [30]:
titles_vectorized_tensor = torch.cat(titles_vectorized)

In [31]:
titles_vectorized_tensor.shape

torch.Size([32818, 1024])

In [32]:
# Сохраним это

In [34]:
torch.save(titles_vectorized_tensor, 'datasets/titles_vectorized.pkl')

In [68]:
# попробуем вычислить дистанции. Сформируем эмбеддинг запроса

In [69]:
encoded_input = tokenizer('химера', padding='max_length', truncation=True, max_length=20, return_tensors='pt')

In [70]:
with torch.no_grad():
    out = model(**encoded_input)

In [71]:
out[1]

tensor([[-0.2461,  0.3397, -0.8128,  ..., -0.2800,  0.8480, -0.1490]])

In [72]:
# Формируем 32К одинаковых тензоров для векторного вычисления парной дистанции

In [73]:
outs = torch.cat(titles_vectorized_tensor.shape[0]*[out[1]])

In [74]:
outs.shape

torch.Size([32818, 1024])

In [75]:
pdist = torch.nn.PairwiseDistance(p=8.0)

In [76]:
dists = pdist(outs, titles_vectorized_tensor).numpy()

In [77]:
dists.shape

(32818,)

In [78]:
np.argmin(dists)

384

In [79]:
# точность
dists[384]

0.0

In [80]:
names[384]

'Химера'

In [81]:
# Победа! )

## Обернем все в функции

In [184]:
def match(sentence, pdist=torch.nn.PairwiseDistance(p=8.0)):
    encoded_input = tokenizer(sentence, max_length=20, truncation=True, 
                   padding='max_length', return_tensors='pt')
    
    with torch.no_grad():
        out = model(**encoded_input)
    
    populated_result = torch.cat(titles_vectorized_tensor.shape[0]*[out[1]])
    distances = pdist(populated_result, titles_vectorized_tensor).numpy()
    length = np.min(distances)
    index = np.argmin(distances)
    return index, length

def variants(sentence):
    result = []
    splitted = sentence.split()
    for i in range(len(splitted)):
        left_slice = splitted[i:]
        for j in range(i+1, len(left_slice) + i + 1):
            result.append(splitted[i:j])
    return result
    
def match_sentence(sentence, lemmer=pymorphy2.MorphAnalyzer()):
    perms = variants(sentence)
    best_match, best_score = -1, 999999
    for var in perms:
        processed = [lemmer.parse(token)[0].normal_form for token in var] if lemmer else var
        text = ' '.join(processed)
        index, length = match(text.lower())
        if length < best_score:
            best_match = index
            best_score = length
    
    print(f'{names[best_match]}, {best_score}')

In [194]:
%%time
match_sentence('Химера')

Химера, 0.0
CPU times: user 3.27 s, sys: 322 ms, total: 3.59 s
Wall time: 269 ms


In [187]:
%%time
match_sentence('Сколько оттяжек нужно на Химере')

Химера, 0.0
CPU times: user 38 s, sys: 5.08 s, total: 43.1 s
Wall time: 3.29 s


In [188]:
%%time
match_sentence('Какая категория у химии любви')

Химия любви, 0.0
CPU times: user 38 s, sys: 4.88 s, total: 42.9 s
Wall time: 3.27 s


In [193]:
%%time
match_sentence('Сколько питчей у мисячне сяйва')

Мисячне сяйво, 0.6415832042694092
CPU times: user 38.2 s, sys: 5.03 s, total: 43.3 s
Wall time: 3.28 s


In [195]:
%%time
match_sentence('расскажи мне о том, Какая категория у химии любви')

Химия любви, 0.0
CPU times: user 1min 52s, sys: 15.1 s, total: 2min 7s
Wall time: 9.76 s


### Теперь комбинируем с SQuAD

In [202]:
def match(sentence, pdist=torch.nn.PairwiseDistance(p=8.0)):
    encoded_input = tokenizer(sentence, max_length=20, truncation=True, 
                   padding='max_length', return_tensors='pt')
    
    with torch.no_grad():
        out = model(**encoded_input)
    
    populated_result = torch.cat(titles_vectorized_tensor.shape[0]*[out[1]])
    distances = pdist(populated_result, titles_vectorized_tensor).numpy()
    length = np.min(distances)
    index = np.argmin(distances)
    return index, length

def variants(sentence):
    result = []
    splitted = sentence.split()
    for i in range(len(splitted)):
        left_slice = splitted[i:]
        for j in range(i+1, len(left_slice) + i + 1):
            result.append(splitted[i:j])
    return result
    
def match_sentence(sentence, lemmer=pymorphy2.MorphAnalyzer()):
    perms = variants(sentence)
    best_match, best_score = -1, 999999
    for var in perms:
        processed = [lemmer.parse(token)[0].normal_form for token in var] if lemmer else var
        text = ' '.join(processed)
        index, length = match(text.lower())
        if length < best_score:
            best_match = index
            best_score = length
    
    return best_match, best_score

def answer(question):
    index, score = match_sentence(question)
    answer = squad_model([text[index]], [question])
    print(f'Маршрут: "{names[index]}"')
    print(answer[0][0])

In [203]:
answer('Химия Любви')

Маршрут: "Химия любви"
Спорт  6c


In [204]:
answer('Где находится Химия Любви')

Маршрут: "Химия любви"
в Крым Симеиз


In [205]:
answer('сколько метров веревки нужно на Химии Любви')

Маршрут: "Химия любви"
сколько нужно оттяжек, нет информации о том, сколько


In [209]:
answer('сколько оттяжек нужно на химере')

Маршрут: "Химера"
6


In [212]:
answer('где находится маршрут химера')

Маршрут: "Химера"
в Крым Никита D. Юг


In [214]:
answer('где находится кадавр')

Маршрут: "Кадавр"
в Крым Красный Камень C


In [215]:
answer('маршрут кадавр')

Маршрут: "Кадавр"
Спорт  7c 14м 9 болтов


In [217]:
answer('сколько веревки нужно на кадавре')

Маршрут: "Кадавр"
14 метров


In [218]:
answer('сколько оттяжек нужно на кадавре')

Маршрут: "Кадавр"
9


In [222]:
answer('какая станция на солнце над типи')

Маршрут: "Солнце над Типи"
Карабин


In [223]:
answer('автострада')

Маршрут: "Автострада"
Спорт  6b 20м 13 болтов, станция Карабин


In [224]:
answer('где находится автострада')

Маршрут: "Автострада"
в Крым Симеиз


In [225]:
answer('сколько пролазов на автостраде')

Маршрут: "Автострада"
69


In [226]:
answer('кому нравится автострада')

Маршрут: "Автострада"
69 людям


In [228]:
answer('never kiss a dog')

Маршрут: "Never kiss a dog"
Спорт  7a 17м 8 болтов


In [229]:
answer('где находится never kiss a dog')

Маршрут: "Never kiss a dog"
в Antalya Geyikbayırı Gizmo 1


In [245]:
answer('курцы умирают рано')

Маршрут: "Сиртаки"
1 людей пролезло


In [246]:
answer('геликоптер')

Маршрут: "Геликоптер"
Спорт  7c 21м 10 болтов, станция Карабин


In [248]:
answer('где находится геликоптер')

Маршрут: "Геликоптер"
в Крым Никита B


In [249]:
answer('где находится розовый фламинго')

Маршрут: "Розовая"
в Москва


In [271]:
# Так, одно слово может перебить сочетания.. это нехорошо. будем ранжировать с учетом сочетаний слов
# У подходящих сочетаниц приоритет выше чем у подходящих единичных слов

In [82]:
def match(sentence, pdist=torch.nn.PairwiseDistance(p=8.0)):
    encoded_input = tokenizer(sentence, max_length=20, truncation=True, 
                   padding='max_length', return_tensors='pt')
    
    with torch.no_grad():
        out = model(**encoded_input)
    
    populated_result = torch.cat(titles_vectorized_tensor.shape[0]*[out[1]])
    distances = pdist(populated_result, titles_vectorized_tensor).numpy()
    length = np.min(distances)
    index = np.argmin(distances)
    return index, length

def variants(sentence):
    result = []
    splitted = sentence.split()
    for i in range(len(splitted)):
        left_slice = splitted[i:]
        for j in range(i+1, len(left_slice) + i + 1):
            result.append(splitted[i:j])
    return result
    
def match_sentence(sentence, lemmer=pymorphy2.MorphAnalyzer()):
    perms = variants(sentence)
    best_match, best_score = -1, (999999, 0)
    for var in perms:
        processed = [lemmer.parse(token)[0].normal_form for token in var] if lemmer else var
        text = ' '.join(processed)
        index, length = match(text.lower())
        
        if length < best_score[0]:
            best_match = index
            best_score = (length, len(processed))
        elif length == best_score[0]:
            if len(processed) > best_score[1]:
                best_match = index
                best_score = (length, len(processed))
    
    return best_match, best_score

def answer(question):
    index, score = match_sentence(question)
    answer = squad_model([text[index]], [question])
    print(f'Маршрут: "{names[index]}"')
    print(answer[0][0])

In [83]:
answer('где находится розовый фламинго')

Маршрут: "Розовый фламинго"
в Крым Никита F


In [84]:
answer('сколько веревки нужно на розовом фламинго')

Маршрут: "Розовый фламинго"
12 метров


In [90]:
%%time
answer('сколько веревки нужно на розовом крымском гекконе')

Маршрут: "Крымский геккон"
12 метров
CPU times: user 42 s, sys: 5.08 s, total: 47.1 s
Wall time: 6.71 s


In [105]:
torch.set_num_threads(4)

In [106]:
%%time
answer('сколько веревки нужно на розовом крымском гекконе')

Маршрут: "Крымский геккон"
12 метров
CPU times: user 28.8 s, sys: 3.43 s, total: 32.3 s
Wall time: 8.33 s


In [86]:
answer('где находится шёпот в ночи')

Маршрут: "Шепот в ночи"
в Крым Симеиз


In [87]:
answer('маршрут шёпот в ночи')

Маршрут: "Шепот в ночи"
Спорт  6a+/7a 26м 15 болтов


In [88]:
answer('какая станция на маршруте голубые ели')

Маршрут: "Голубые ели"
Цепь


In [89]:
squad_model([text[0]], ['крымский геккон'])

[[''], [-1], [0.4366404414176941]]